# Introduction

Welcome to this tutorial on using jupyter notebooks with Moveit 2. A great benefit of being able to interact with MoveIt via a Python notebook is the ability to rapidly prototype code. We hope you find this interface intuitive and that you gain value from using MoveIt via Python notebooks.

In this tutorial we will cover the following: 

* The required imports to run the notebook
* A motion planning example
* A teleoperation example

If you have suggestions or feedback for this tutorial please post an issue on GitHub (https://github.com/ros-planning/moveit2_tutorials) and tag @peterdavidfagan.

## Imports

Note: to launch this notebook and the nodes it depends on you must first specify a launch file. Details are provided earlier in this tutorial ().

In [1]:
import os
import math
import sys
import yaml
import rclpy
import numpy as np

# message libraries
from geometry_msgs.msg import PoseStamped, Pose

# moveit_py
from moveit.planning import (
    MoveItPy,
    PlanRequestParameters,
    MultiPipelinePlanRequestParameters,
)
from moveit.core.robot_state import RobotState
from moveit.core.kinematic_constraints import construct_joint_constraint

# config file libraries
from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory


# we need to specify our moveit_py config at the top of each notebook we use. 
# this is since we will start spinning a moveit_py node within this notebook.

moveit_config = (
            MoveItConfigsBuilder(robot_name="franka_panda", package_name="moveit_resources_franka_panda_moveit_config")
            .robot_description(file_path=get_package_share_directory("moveit_resources_franka_panda_description") + "/urdf/panda_arm.urdf.xacro", 
                mappings={"robot_ip": "192.168.106.39", "hand": "true"})
            .robot_description_semantic("config/franka_panda.srdf")
            .trajectory_execution("config/moveit_controllers.yaml")
            .moveit_cpp(file_path=get_package_share_directory("panda_moveit_demos") + "/config/notebook.yaml")
            .to_moveit_configs()
            ).to_dict()

## Setup

In [2]:
# initialise rclpy (only for logging purposes)
rclpy.init()

# instantiate moveit_py instance and a planning component for the panda_arm
moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
panda = moveit.get_planning_component("panda_arm")

[INFO] [1677079979.670386861] [moveit_cpp_initializer]: Initialize rclcpp
[INFO] [1677079979.670454021] [moveit_cpp_initializer]: Initialize node parameters
[INFO] [1677079979.698121669] [moveit_cpp_initializer]: params_filepath: /tmp/launch_params_7ucbgu_l
[INFO] [1677079979.698142849] [moveit_cpp_initializer]: Initialize node and executor
[INFO] [1677079979.756566928] [moveit_cpp_initializer]: Spin separate thread
[INFO] [1677079979.759319326] [moveit_rdf_loader.rdf_loader]: Loaded robot model in 0.00261219 seconds
[INFO] [1677079979.759387906] [moveit_robot_model.robot_model]: Loading robot model 'panda'...
[WARN] [1677079979.765991537] [moveit_robot_model.robot_model]: Link panda_leftfinger has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[WARN] [1677079979.766137068] [moveit_robot_model.robot_model]: Link panda_rightfinger has visual geometry but no collision geometry. Collision ge

## Motion Planning Example

In [3]:
# set plan start state using predefined state
panda.set_start_state_to_current_state()

# set goal using a pose message this time
panda.set_goal_state(configuration_name="ready")

# plan to goal
planner_params = PlanRequestParameters(moveit, "pilz_lin")
plan_result = panda.plan(single_plan_parameters=planner_params)

[INFO] [1677079980.816200908] [moveit.pilz_industrial_motion_planner.trajectory_generator_ptp]: Initialized Point-to-Point Trajectory Generator.
[INFO] [1677079980.816305368] [moveit.pilz_industrial_motion_planner.trajectory_generator]: Generating PTP trajectory...


In [4]:
# execute the plan
if plan_result:
    robot_traj = plan_result.trajectory
    moveit.execute(robot_traj, blocking=True, controllers=[])

[INFO] [1677079981.379178445] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1677079981.379240505] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1677079981.379381825] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1677079981.379409115] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1677079981.379530356] [moveit_ros.trajectory_execution_manager]: Validating trajectory with allowed_start_tolerance 0.01
[INFO] [1677079981.413273783] [moveit_ros.trajectory_execution_manager]: Starting trajectory execution ...
[INFO] [1677079981.413321984] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1677079981.413363024] [moveit.plugins.moveit_simple_controller_manager]: Returned 2 controllers in list
[INFO] [1677079981.413544404] [moveit.simple_controller_manager.follow_joint_trajectory_cont

In [7]:
# set plan start state using predefined state
panda.set_start_state_to_current_state()

# set goal using a pose message this time
pose_goal = PoseStamped()
pose_goal.header.frame_id = "panda_link0"
pose_goal.pose.orientation.w = 1.0
pose_goal.pose.position.x = 0.28
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = 0.5
panda.set_goal_state(pose_stamped_msg = pose_goal, pose_link = "panda_link8")

# set pose goal using predefined state
#panda.set_goal_state(configuration_name="extended")



# plan to goal
planner_params = PlanRequestParameters(moveit, "pilz_lin")
plan_result = panda.plan(single_plan_parameters=planner_params)

In [8]:
# execute the plan
if plan_result:
    robot_traj = plan_result.trajectory
    moveit.execute(robot_traj, blocking=True, controllers=[])

# Teleoperation

In [4]:
from moveit.servo_client.devices.ps4_dualshock import PS4DualShockTeleop


ps4 = PS4DualShockTeleop(ee_frame_name="panda_link8")
ps4.start_teleop()

In [ ]:
ps4.stop_teleop()